In [1]:
import random
import datetime
import numpy as np
import h5py
from skimage.filters import threshold_otsu
from skimage.transform import rescale, resize
from skimage.draw import polygon as ski_polygon
from skimage.morphology import dilation
from skimage import io
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import cm
import math
import tifffile
import os
import sys
import shutil
import json
import calculate_performance as calc

import openslide
from preprocessing.datamodel import SlideManager
from preprocessing.processing import split_negative_slide, split_positive_slide, create_tumor_mask, create_tumor_outline, rgb2gray
from preprocessing.util import TileMap

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_curve, auc
from scipy.special import softmax #e to the x and divide by sum
import matplotlib.pyplot as plt
import time
import copy
from tqdm import tqdm
import shutil

%matplotlib inline

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.6.0
Torchvision Version:  0.7.0


In [2]:
def heatmap_overlay(slide, heatmap, mask, falsecolored, max_px, truncate):
    resize_factor = max_px/max(heatmap.shape)
    
    ## Downsample images to save memory
    heatmap = rescale(heatmap, resize_factor, anti_aliasing = False, preserve_range = 'True')
    mask = rescale(mask, resize_factor, anti_aliasing = False)
    
    fc_PIL = Image.fromarray(falsecolored) ## Convert to PIL
    fc_PIL = fc_PIL.resize((heatmap.shape[1], heatmap.shape[0])) ## Resize to be same size as downsampled heatmap

    [r,c] = np.where(heatmap < truncate) ## Grab coordinates heatmap values < truncate (i.e. background value)

    ## Define colormap
    cmap_trunc = 0.45 ## Adjust colormap (if you want to display yellow to red, rather than yellow-->red-->purple)
    heatmap_PIL = cm.magma(heatmap*cmap_trunc + (1-cmap_trunc)) ## Apply desired colormap
    heatmap_PIL = Image.fromarray((heatmap_PIL*255).astype('uint8')) ## Format array for PIL then convert to PIL

    ## Define opacity mask for false colored image during heatmap blending step. Want hotspot heatmap areas (~1) to have zero falsecolored opacity (0)
    cmap_opacity = 1.5 ## Default = 1. Rescale colormap to increase its opacity
    fc_opacity = ((1-heatmap)*255/cmap_opacity).astype('uint8') ## Apply rescaling factor to bring out heatmap
    fc_opacity[r,c] = 255 ## Then set falsecolor opacity to white wherever heatmap is < threshold
    fc_opacity = Image.fromarray(fc_opacity)

    ## Overlay the false-colored image and heatmap
    heatmap_fc = Image.composite(fc_PIL, heatmap_PIL, fc_opacity) #fc_opacity of 255 (or 1 if bool) --> fc image only

    ## Overlay an outline of ground-truth annotation onto image (optional)
    mask_outline = create_tumor_outline(slide, level = 3)
    kernel = np.ones((5,5), np.uint8) 
    mask_outline = dilation(mask_outline, kernel) ## Give mask outline a line thickness (display purposes)

    ## Ensure mask array is the same size as heatmap array
    mask_outline = resize(mask_outline, output_shape = (heatmap.shape[0], heatmap.shape[1]), anti_aliasing = False, preserve_range = 'True')
    mask_outline = 1 - mask_outline ## Invert values so that the annotation appears black
    [r,c] = np.where(mask_outline == 0) ## Grab pixel locations for the annotation
    heatmap_fc_gt = np.array(heatmap_fc)
    heatmap_fc_gt[r,c,:3] = 0 ## Assign the annotation pixels as black

    return heatmap_fc, heatmap_fc_gt

## Load slide manager and parameters

In [3]:
DIR = ''

mgr = SlideManager(cam16_dir=DIR)

slides_met = mgr.met_slides

N_met = len(slides_met)

slides_negative = mgr.negative_slides

N_negative = len(slides_negative)

level = 0

tile_size = 512 #must be the same as used for training

poi = 0.50 #must use the same poi we used to seperate tisse from background

overlap = tile_size // 2 #increasing overlap will put patches closer together

In [4]:
data_dir = "test//"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

prediction_folder_path = 'predictions//'

## Generate heatmaps for OTLS en face images ("slides")

In [5]:
## Go through each slide, find corresponding trained model, and generate heatmap
for i in range(len(mgr.slides)):
    slide = mgr.slides[i] ## Define which slide to generate predictions on 
    case_name = slide.name.split('_')[0] ## Grab case name from file name - relies on provided naming convention format
    case_number = case_name[-2:] ## Conver the last two digits to integer

    model_name = 'fold' + str(case_number) + '_model.pt'
    model = torch.load(model_name)
    model.eval()
    print('Making predictions for ' + str(case_name))
    print('Model name = ' + str(model_name))

    input_size = 512 # Define patch size in pixels 
    ## Compute threshold we will use to define tissue regions using Otsu
    arr = np.asarray(slide.get_full_slide(level=4))
    arr_gray = rgb2gray(arr)
    threshold = threshold_otsu(arr_gray)
    
    # Get ground truth label (2D mask)
    mask = create_tumor_mask(slide, level=level)
    
    ## Get false-colored image (for visualization purposes)
    fc_im = io.imread('images//false-colored//' + str(slide.name) + '.tiff')

    # #Initialize heatmap
    slide_level0 =  np.asarray(slide.get_full_slide(level = 0))[:,:,:3]
    size = slide.level_dimensions[level]

    # Load patch/tile iterator
    tile_iter = split_negative_slide(
        slide, level=level,
        otsu_threshold=threshold,  # otsu threshold calculated earlier
        tile_size=tile_size,
        overlap=overlap,                 # overlap between patches
        poi_threshold=poi,
        verbose = False)         # only select tiles with at least 5% tissue

    ## Initialize variables     
    all_outputs_normalized = None ## Will be a 2D array of all patch-based predictions (column 0 = P(0), column 1 = P(1))
    Ptumor = None
    P = None ## Probability that a single patch is neoplastic
    all_labels = None

    heatmap_fullres = np.zeros((size[1] + tile_size, size[0] + tile_size), dtype = 'float64') # Initialize heatmap array
    division_map = np.zeros((size[1] + tile_size, size[0] + tile_size),dtype = 'float64') # Initialize map that will help us account for duplicate predictions
    
    ii = 0
    for patch, bounds in tile_iter: #bounds is (X,Y),(width,height)
        ## Get coordinates of patch on full res slide
        #Assumes start_pos = (0,0)
        X = bounds[0][0] #X coordinate of top left corner on full-resolution slide (not downsampled by level)
        Y = bounds[0][1] #Y coordinate of top left corner on full-resolution slide
        width = bounds[1][0] #width of patch
        height = bounds[1][1] #height of patch

        ## Perform same preprocessing that is done on patches
        patch = patch.astype('float64')
        patch[:,:,0] = patch[:,:,0]*255/patch[:,:,0].max()
        patch[:,:,1] = patch[:,:,1]*255/patch[:,:,1].max()

        #Format patch and predict probability of having neoplasia
        patch = np.moveaxis(patch, 2, 0)
        patch = np.expand_dims(patch,0)
        patch = np.float32(patch) #tensors loaded by dataloader are float32
        patch[:,0,:,:] = patch[:,0,:,:]/patch[:,0,:,:].max() # Normalize channel 0 to its max
        patch[:,1,:,:] = patch[:,1,:,:]/patch[:,1,:,:].max() # Normalize channel 1 to its max

        #Perform same operation as transforms. Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        patch[:,0,:,:] = (patch[:,0,:,:] - .485)/.229
        patch[:,1,:,:] = (patch[:,1,:,:] - .456)/.224
        patch[:,2,:,:] = (patch[:,2,:,:] - .406)/.225

        patch = torch.from_numpy(patch)
        patch = patch.to(device)
        output = model(patch)
        output_normalized = softmax(output.cpu().detach().numpy(),axis = 1) #array of size (2,1)
        P = output_normalized[0][1] #extracts JUST the probability for class 1

        ##GROUND TRUTH
        ## Get coordinates of patch on slide downsampled by level
        r = np.array((Y, Y, Y+height, Y+height), dtype=np.float32)
        r /= round(slide.level_downsamples[level]) # will != 1 if downsampling is applied
        r = np.array(r + 0.5, dtype=np.int32) # converts to integer (rounds decimals down)
        c = np.array((X, X+width, X+width, X), dtype=np.float32)
        c /= round(slide.level_downsamples[level]) #will != 1 if downsampling is applied
        c = np.array(c + 0.5, dtype=np.int32) # converts to integer (rounds decimals down)
        rr, cc = ski_polygon(r, c, shape=mask.shape) ##patch coords of TISSUE

        ## Assign heatmap values 
        heatmap_fullres[r[0]:r[3], c[0]:c[1]] += P
        division_map[r[0]:r[3], c[0]:c[1]] += 1 ## Keep track of how many times we have predicted on this patch (and added to the heatmap)

        ## Get ground truth label according to mask (downsampled by level)
        if mask[rr,cc].max() == 1:
            label = 1
        else:
            label = 0

        if all_outputs_normalized is None:
            all_outputs_normalized = output_normalized
            all_labels = label
            Ptumor = P
        else:
            all_outputs_normalized = np.append(all_outputs_normalized, output_normalized, axis = 0)
            all_labels = np.append(all_labels, label)
            Ptumor = np.append(Ptumor, P)    

        ii += 1

    ## Divide the heatmap by the division map so that all pixels represent the average of all predictions made on each patch
    division_map[np.where(division_map == 0)] = 1
    heatmap_fullres = heatmap_fullres/division_map
    
    ## Overlay heatmap onto false-colored image
    heatmap_fc, heatmap_fc_gt = heatmap_overlay(slide, heatmap_fullres, mask, fc_im, 1000, 0.10)
    tifffile.imwrite(prediction_folder_path + str(slide.name) + '_heatmap_fc_gt.tiff', heatmap_fc_gt, photometric = 'rgb')

    ## Save the ground truth mask
    slide_mask = np.zeros((size[1], size[0]), dtype = 'uint8')
    r, c = np.where(mask == 1)
    slide_mask[r,c] = 255  #For solid red overlay: slide_mask[r,c,0] = 255 # slide_mask[r,c,1:] = 0
    tifffile.imwrite(prediction_folder_path + str(slide.name) + '_ground truth.tiff', slide_mask, photometric = 'rgb')

    ## Save full-resolution heatmap
    tifffile.imwrite(prediction_folder_path + str(slide.name)+ '_heatmap.tiff', heatmap_fullres.astype('float64'), photometric = 'minisblack')

Making predictions for case00
Model name = fold00_model.pt
Making predictions for case01
Model name = fold01_model.pt
Making predictions for case00
Model name = fold00_model.pt
Making predictions for case01
Model name = fold01_model.pt
